#**Sentiment Analysis on Movie Reviews 電影評論情緒分類**
*   0 → negative 負面
*   1 → Somewhat negative  一點點負面
*   2 → Neutral 一般
*   3 → Somewhat positive 一點點正面
*   4 → Positive 正面










### pytorch環境設定(https://pytorch.org)
選擇適合的版本並複製指令貼到本專案

In [1]:
!pip install torch torchvision

In [2]:
#確定有安裝成功
import torch
print(torch.__version__)
torch.cuda.is_available()

1.4.0


True

### **取得 google drive 存取權限**

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
cd /content/drive/My Drive/dataset/Sentiment Analysis(MovieReviews)/

/content/drive/My Drive/dataset/Sentiment Analysis(MovieReviews)


In [5]:
!ls

model  predict.ipynb  train.ipynb


In [0]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### **下載dataset**

In [7]:
import os
def fileExists(filepath):
  return  os.path.exists(filepath)
#我把資料集放到我的github
if fileExists('train.tsv')!=True:
  !wget https://raw.githubusercontent.com/harry83528/sentiment-analysis-on-movie-reviews/master/data/train.tsv

--2020-04-11 16:37:50--  https://raw.githubusercontent.com/harry83528/sentiment-analysis-on-movie-reviews/master/data/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8481022 (8.1M) [text/plain]
Saving to: ‘train.tsv’

train.tsv           100%[===================>]   8.09M  38.5MB/s    in 0.2s    

2020-04-11 16:37:51 (38.5 MB/s) - ‘train.tsv’ saved [8481022/8481022]



In [8]:
!ls

model  predict.ipynb  train.ipynb  train.tsv


In [9]:
import os
import pandas as pd
df_train = pd.read_csv("train.tsv", sep='\t')
df_train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [10]:
df_train.shape

(156060, 4)

In [11]:
df_train.columns

Index(['PhraseId', 'SentenceId', 'Phrase', 'Sentiment'], dtype='object')

In [12]:
df_train.index

RangeIndex(start=0, stop=156060, step=1)

In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156060 entries, 0 to 156059
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   PhraseId    156060 non-null  int64 
 1   SentenceId  156060 non-null  int64 
 2   Phrase      156060 non-null  object
 3   Sentiment   156060 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 4.8+ MB


In [14]:
df_train.describe()

,PhraseId,SentenceId,Sentiment
count,156060.000000,156060.000000,156060.000000
mean,78030.500000,4079.732744,2.063578
std,45050.785842,2502.764394,0.893832
min,1.000000,1.000000,0.000000
25%,39015.750000,1861.750000,2.000000
50%,78030.500000,4017.000000,2.000000
75%,117045.250000,6244.000000,3.000000
max,156060.000000,8544.000000,4.000000


### **將訓練資料tsv檔轉成txt檔(【\t】分隔欄位)**



In [0]:
# 將要訓練的句子存起來
def SaveSentence(filepath, sent_list):
    f = open(filepath, 'w', encoding='UTF-8')
    for sent in sent_list:
        f.write(sent + '\n')
    f.close()

# 分割train_data和test_data
def data_Split(FileName):

    fp = open(FileName, 'r', encoding='utf-8')
    line = fp.readline()        # 第一行是表頭PhraseId,SentenceId,Phrase,Sentiment
    line = fp.readline()        # 實際資料的第一行

    test_data = []

    # 用 while 逐行讀取檔案內容(一次讀一行)，直至檔案結尾
    while line:
        sent = ''
        sent = line.replace('\n', '')
        test_data.append(sent)
        line = fp.readline()  #位移到下一行
    
    fp.close()
    
    SaveSentence('train_data.txt', test_data)

In [0]:
if fileExists('train_data.txt')!=True:
  data_Split('train.tsv')

### **安裝所需的函式庫-HuggingFace 團隊將 GitHub 專案大翻新並更名成 transformers(Install the Hugging Face Library)**
https://github.com/huggingface/transformers

In [17]:
!pip install transformers

     |████████████████████████████████| 573kB 4.8MB/s 
     |████████████████████████████████| 3.7MB 15.9MB/s 
     |████████████████████████████████| 1.0MB 55.3MB/s 
     |████████████████████████████████| 870kB 61.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=732417ae8d8213079aac0cebd6c6b48c8c9727e20893c0dcdd1b523deffc37de
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
%tensorflow_version 2.x

### **指定想要用的模型是哪一種**
到以下網址查詢模型與函數資訊
https://huggingface.co/transformers

In [0]:
PRETRAINED_MODEL_Folder ="model/"
PRETRAINED_MODEL_Name ="albert-base-v1-pytorch_model.bin"  # 指定 想要用的預訓練模型
PRETRAINED_Spiece_Name ="albert-base-spiece.model"
PRETRAINED_Config_Name ="albert-base-config.json"  #唯獨這個檔案自己下載好上傳並修改設定檔參數和增加num_labels=5
PRETRAINED_MODEL_Path =PRETRAINED_MODEL_Folder + PRETRAINED_MODEL_Name
PRETRAINED_Spiece_Path =PRETRAINED_MODEL_Folder + PRETRAINED_Spiece_Name
PRETRAINED_Config_Path =PRETRAINED_MODEL_Folder + PRETRAINED_Config_Name
trained_Model_Path = "trained_model"

### **下載想要的模型**

In [0]:
#創⼀個資料夾存bert預訓練模型
if not os.path.isdir(PRETRAINED_MODEL_Folder):
    os.mkdir(PRETRAINED_MODEL_Folder)

In [0]:
import requests

# 下載必要的模型檔案
# Download needed model files

urls = ['https://s3.amazonaws.com/models.huggingface.co/bert/'+PRETRAINED_MODEL_Name,
        'https://s3.amazonaws.com/models.huggingface.co/bert/'+PRETRAINED_Spiece_Name]
filenames = [PRETRAINED_MODEL_Name,PRETRAINED_Spiece_Name]

for i,url in enumerate(urls):
    r = requests.get(url)
    with open(PRETRAINED_MODEL_Folder+filenames[i], 'wb') as f:
        f.write(r.content)

## **引入需要⽤到的函式庫**

In [0]:
import os
import pickle
import torch
#from transformers import BertConfig, BertTokenizer, BertForSequenceClassification, AdamW
from transformers import AlbertConfig,AlbertTokenizer,AlbertForSequenceClassification,AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F # 激勵函數

### **載入資料並將資料轉換成輸入格式(Token Embeddings、Segment Embeddings、Position Embeddings)**

In [0]:
def convert_data_to_feature(FileName):
    # 載入字典
    #tokenizer = BertTokenizer(vocab_file='bert-base-chinese-vocab.txt')
    #tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
    tokenizer = AlbertTokenizer.from_pretrained(PRETRAINED_Spiece_Path)

    # 載入資料
    Labels = []
    Sentences = []
    SentencesId = []
    with open(FileName,'r',encoding='utf-8') as f:
        data = f.read()
    LS_pairs = data.split("\n")
    print('長度:',len(LS_pairs))
    #因為 label 與sentence在同⼀筆資料，所以我們將他們分開並個別存起來
    for LS_pair in LS_pairs:
        if LS_pair != "":
            try:
                #print(LS_pair)#2  1 A series of escapades demonstrating the adage that what is good for the goose 2
                LS_ary = LS_pair.split("\t")  #用tsv的【\t】來分割欄位
                SId=LS_ary[0] #句子id        
                S = LS_ary[2] #句子
                L = LS_ary[3] #情緒label
                SentencesId.append(int(SId))  
                Sentences.append(S)
                Labels.append(int(L))
            except:
                continue
    
    assert len(Labels) == len(Sentences)  == len(SentencesId)
    print('Sentences:',Sentences[0:20])
    # BERT input embedding
    max_seq_len = 0     # 紀錄最大長度
    input_ids = []
    original_length = []    # 紀錄原本長度
    #==轉換成BERT input 需要的格式===============================================
    for S in Sentences:
        # 將句子切割成一個個token
        word_piece_list = tokenizer.tokenize(S)
        # 將token轉成字典中的id
        input_id = tokenizer.convert_tokens_to_ids(word_piece_list)
        # 補上[CLS]和[SEP]
        input_id = tokenizer.build_inputs_with_special_tokens(input_id)

        if(len(input_id)>max_seq_len):
            max_seq_len = len(input_id)
        input_ids.append(input_id)
        #反向查詢(還原)
        #print(tokenizer.convert_ids_to_tokens(tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(input_id)))))
    #==轉換成BERT input 需要的格式  END==========================================
    
    print("最長句子長度:",max_seq_len)
    
    assert max_seq_len <= 512 # 小於BERT-base長度限制
    max_seq_len = 512

    #統⼀輸入資料的長度==========================================================
    # 補齊長度
    for c in input_ids:
        # 紀錄原本長度
        length = len(c)
        original_length.append(length)
        while len(c)<max_seq_len:
            c.append(0)
    #⽣成每⼀句的type_id
    token_type_ids = [[0]*max_seq_len for i in range(len(Sentences))]        # token_type_ids # 儲存的是句子的id，id為0就是第一句，id為1就是第二句
    attention_mask = []                                                      # attention_mask # 1代表是真實的單詞id，0代表補齊長度
    for i in range(len(Sentences)):
        attention_id = []
        for j in range(original_length[i]):
            attention_id.append(1)
        while len(attention_id)<max_seq_len:
            attention_id.append(0)
        attention_mask.append(attention_id)

    assert len(input_ids) == len(token_type_ids) and len(input_ids) == len(attention_mask) and len(input_ids) == len(Labels) and len(input_ids) == len(SentencesId)

    #為了使⽤⽅便，把三個input embedding及 label 包成 object======================
    data_features = {'input_ids':input_ids,
                    'token_type_ids':token_type_ids,
                    'attention_mask':attention_mask,
                    'labels':Labels,
                    'sentences_ids':SentencesId}

    print('sentences_ids:',SentencesId[0:10])
    print('input_ids:',input_ids[0:10])
    print('Labels:',Labels[0:50])
    return data_features

### **輸入格式轉Dataset**

In [0]:
def makeDataset(data_feature):
    input_ids = data_feature['input_ids']
    token_type_ids = data_feature['token_type_ids']
    attention_mask = data_feature['attention_mask']
    labels = data_feature['labels']
    sentences_ids = data_feature['sentences_ids']

    all_input_ids = torch.tensor([input_id for input_id in input_ids], dtype=torch.long)
    all_token_type_ids = torch.tensor([token_type_id for token_type_id in token_type_ids], dtype=torch.long)
    all_attention_mask_ids = torch.tensor([attention_id for attention_id in attention_mask], dtype=torch.long)
    all_labels = torch.tensor([label for label in labels], dtype=torch.long)
    all_sentences_ids = torch.tensor([sentences_id for sentences_id in sentences_ids], dtype=torch.long)
    dataset = TensorDataset(all_input_ids, all_token_type_ids, all_attention_mask_ids, all_labels ,all_sentences_ids)

    return dataset

## **確定 Embedding 是否正確**

In [25]:
#tokenizer = BertTokenizer(vocab_file='bert-base-chinese-vocab.txt')
# 取得此預訓練模型所使用的 tokenizer
#tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
tokenizer = AlbertTokenizer.from_pretrained(PRETRAINED_Spiece_Path)
data_features = convert_data_to_feature('train_data.txt')

print(data_features['input_ids'][43636])
print(tokenizer.convert_ids_to_tokens(data_features['input_ids'][43636]))
print(data_features['token_type_ids'][43636])
print(data_features['attention_mask'][43636])
print(data_features['labels'][43636])
print(data_features['sentences_ids'][43636])

Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated
Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated


長度: 156061
Sentences: ['A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .', 'A series of escapades demonstrating the adage that what is good for the goose', 'A series', 'A', 'series', 'of escapades demonstrating the adage that what is good for the goose', 'of', 'escapades demonstrating the adage that what is good for the goose', 'escapades', 'demonstrating the adage that what is good for the goose', 'demonstrating the adage', 'demonstrating', 'the adage', 'the', 'adage', 'that what is good for the goose', 'that', 'what is good for the goose', 'what', 'is good for the goose']
最長句子長度: 93
sentences_ids: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
input_ids: [[2, 21, 231, 16, 13, 13944, 8240, 160, 16133, 14, 21, 43, 1303, 30, 98, 25, 254, 26, 14, 13669, 25, 67, 254, 26, 14, 19383, 106, 13, 15, 109, 16, 56, 4533, 21, 25443, 18, 47, 2369, 16, 56, 8545, 20, 212, 16, 2

## **分割數據**
把資料切成訓練集跟測試集
對於 Supervised Learning 來說，我們使⽤標註資料（Labeled Data）來做訓練
因此我們需要訓練資料（Training Data）
但是我們蒐集到的資料卻不能全部拿來做訓練
因為我們必須要保留⼀些當作測試資料（Testing Data）來評估模型表現
這些資料必須跟訓練資料是完全不同的，否則就有作弊的嫌疑
所以我們會把我們的資料切成訓練集與測試集，通常會保留比較多當作訓練資料
（60%~80%），其餘做測試資料

In [0]:
def split_dataset(full_dataset, split_rate=0.8):  
    train_size = int(split_rate * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
    return train_dataset,test_dataset

### **Fine-Tuning**


In [0]:
# 計算正確值
def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

In [28]:
#設定模型參數 Start =============================================================

#bert_config, bert_class, bert_tokenizer = (BertConfig, BertForSequenceClassification, BertTokenizer)


#Step1:初始化加載模型 Start======================================================
config = AlbertConfig.from_json_file(PRETRAINED_Config_Path)  #這步驟非常正要,要指定label數,不然預設是二分類
model = AlbertForSequenceClassification.from_pretrained(PRETRAINED_MODEL_Path, config = config)
#config = bert_config.from_pretrained('bert-base-cased',num_labels = 5)  #這步驟非常正要,要指定label數,不然預設不會是多分類
#model = bert_class.from_pretrained('bert-base-cased', from_tf=bool('.ckpt' in 'bert-base-cased'), config=config)  #這步驟非常正要,要指定label數,不然預設會是2分類
#model = BertForSequenceClassification.from_pretrained('bert-base-cased')
#Step1:初始化加載模型 END========================================================

#Step2:指定硬體裝置 Start========================================================
# setting device
#你電腦的 GPU 能否被 PyTorch 調用,如果不行就使用CPU 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("using device",device)
model.to(device)
#Step2:指定硬體裝置 END==========================================================

#Step3:將訓練資料讀入並且組建BERT輸入格式 Start===================================
train_data_feature = convert_data_to_feature('train_data.txt')
#for i in range(500):
#print(data_features['labels'][i],sep=",")
#test_data_feature = convert_data_to_feature('test_data.txt')
full_dataset = makeDataset(train_data_feature)
#test_dataset = makeDataset(test_data_feature)
 #Step3:將訓練資料讀入並且組建BERT輸入格式 END====================================

#Step4:將組建好的輸入格式轉換成tensor格式，並且建立dataset與dataloader Srart======
train_dataset, test_dataset = split_dataset(full_dataset, 0.8)
train_dataloader = DataLoader(train_dataset ,batch_size=4 ,shuffle=True)
test_dataloader = DataLoader(test_dataset ,batch_size=4 ,shuffle=True)
#Step4:將組建好的輸入格式轉換成tensor格式，並且建立dataset與dataloader END========

#準備優化器 Start===============================================================
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
Learning_rate = 5e-6       # 學習率
optimizer = AdamW(optimizer_grouped_parameters, lr=Learning_rate, eps=1e-8)
#準備優化器 End=================================================================


#設定模型參數 End===============================================================

using device cuda


Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated


長度: 156061
Sentences: ['A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .', 'A series of escapades demonstrating the adage that what is good for the goose', 'A series', 'A', 'series', 'of escapades demonstrating the adage that what is good for the goose', 'of', 'escapades demonstrating the adage that what is good for the goose', 'escapades', 'demonstrating the adage that what is good for the goose', 'demonstrating the adage', 'demonstrating', 'the adage', 'the', 'adage', 'that what is good for the goose', 'that', 'what is good for the goose', 'what', 'is good for the goose']
最長句子長度: 93
sentences_ids: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
input_ids: [[2, 21, 231, 16, 13, 13944, 8240, 160, 16133, 14, 21, 43, 1303, 30, 98, 25, 254, 26, 14, 13669, 25, 67, 254, 26, 14, 19383, 106, 13, 15, 109, 16, 56, 4533, 21, 25443, 18, 47, 2369, 16, 56, 8545, 20, 212, 16, 2

### **開始訓練**

In [29]:
#Step5:建構training loop、開始訓練 Start=========================================
for epoch in range(3):
    
    All_train_correct = 0.0
    AllTrainLoss = 0.0
    count = 0
    model.zero_grad() #梯度歸零
    for batch_dict in train_dataloader:

        # 訓練模式
        model.train()
        
        batch_dict = tuple(t.to(device) for t in batch_dict)
        #print(batch_dict[0].shape)
        #print(batch_dict[1].shape)
        #print(batch_dict[2].shape)
        #print(batch_dict[3].shape)
        #print(batch_dict[4].shape)
        #順序對應產⽣Dataset那裡的dataset = TensorDataset(all_input_ids, all_segment_ids,all_position_ids, all_labels ,all_sentences_ids)
        outputs = model(
            input_ids = batch_dict[0],
            token_type_ids = batch_dict[1],             
            attention_mask = batch_dict[2],             
            labels = batch_dict[3]
            )
        loss, logits = outputs[:2]
            
        train_correct = compute_accuracy(logits, batch_dict[3])       # 計算正確率
        All_train_correct += train_correct
        AllTrainLoss += loss.item()
        count += 1

        model.zero_grad()
        loss.backward()
        optimizer.step()
            
    Average_train_correct = round(All_train_correct/count, 3)
    Average_train_loss = round(AllTrainLoss/count, 3)

    # 測試模式
    model.eval()
    All_test_correct = 0.0
    AllTestLoss = 0.0
    count = 0
    for batch_dict in test_dataloader:
        batch_dict = tuple(t.to(device) for t in batch_dict)  #每⼀個Batch把資料從CPU RAM 推到 GPU RAM

        outputs = model(
            input_ids = batch_dict[0],
            token_type_ids = batch_dict[1],            
            attention_mask = batch_dict[2],           
            labels = batch_dict[3]
            )
        loss, logits = outputs[:2]

        test_correct = compute_accuracy(logits, batch_dict[3])       # 計算正確率
        All_test_correct += test_correct
        AllTestLoss += loss.item()

        count += 1
        
    Average_test_correct = round(All_test_correct/count, 3)
    Average_test_loss = round(AllTestLoss/count, 3)

    print('第' + str(epoch+1) + '次' + '訓練模式，loss為:' + str(Average_train_loss) + ' 正確率為' + str(Average_train_correct)+ '，測試模式，loss為:' + str(Average_test_loss) + ' 正確率為' + str(Average_test_correct))

#Step5:建構training loop、開始訓練 END===========================================

第1次訓練模式，loss為:0.887 正確率為63.419，測試模式，loss為:0.81 正確率為65.895
第2次訓練模式，loss為:0.77 正確率為67.945，測試模式，loss為:0.765 正確率為68.39
第3次訓練模式，loss為:0.71 正確率為70.507，測試模式，loss為:0.745 正確率為69.262


### **儲存模型**

In [0]:
#創⼀個資料夾存模型
if not os.path.isdir(trained_Model_Path):
    os.mkdir(trained_Model_Path)
#儲存
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(trained_Model_Path)

In [0]:
#predict 時把輸入的⽂字轉成輸入格式
def to_input_id(sentence_input):
    return tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence_input)))

In [32]:
# load and init
#將輸入的句⼦轉為wordpices格式並載入我們train好的模型
#tokenizer = BertTokenizer(vocab_file='bert-base-chinese-vocab.txt')
# 取得此預訓練模型所使用的 tokenizer
tokenizer = AlbertTokenizer.from_pretrained(PRETRAINED_Spiece_Path)

config = AlbertConfig.from_json_file(trained_Model_Path+'/config.json') #預測時用訓練產出的設定檔
#config = AlbertConfig.from_pretrained(PRETRAINED_MODEL_Path,num_labels = 5)  #這步驟非常正要,要指定label數,不然預設是二分類
model = AlbertForSequenceClassification.from_pretrained(trained_Model_Path+'/pytorch_model.bin', config = config) #預測時用訓練產出的模型


model.eval()

print('請輸入句子')
sentence = 'good'

input_id = to_input_id(sentence)
assert len(input_id) <= 512

while len(input_id)<512:
    input_id.append(0)

input_ids = torch.LongTensor(input_id).unsqueeze(0)

# predict時，因為沒有label所以沒有loss
outputs = model(input_ids)
print('1',outputs,type(outputs))
predicts = outputs[:2]
print('2',predicts,type(predicts))
predicts = predicts[0]
print('3',predicts,type(predicts))
max_val = torch.max(predicts)
print('4',max_val,type(max_val))
predict_label = (predicts == max_val).nonzero().numpy()[0][1] # 在第1維度取最大值並返回索引值

print('label:'+str(predict_label))

if str(predict_label) == '0':
  print('Negative')
elif str(predict_label) == '1':
  print('Somewhat negative')
elif str(predict_label) == '2':
  print('Neutral')
elif str(predict_label) == '3':
  print('Somewhat positive')
elif str(predict_label) == '4':
  print('Positive')
else:
  print('程式發生錯誤')

Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated


請輸入句子
1 (tensor([[-2.2407, -1.5903,  0.8635,  1.5410,  1.4544]],
       grad_fn=<AddmmBackward>),) <class 'tuple'>
2 (tensor([[-2.2407, -1.5903,  0.8635,  1.5410,  1.4544]],
       grad_fn=<AddmmBackward>),) <class 'tuple'>
3 tensor([[-2.2407, -1.5903,  0.8635,  1.5410,  1.4544]],
       grad_fn=<AddmmBackward>) <class 'torch.Tensor'>
4 tensor(1.5410, grad_fn=<MaxBackward1>) <class 'torch.Tensor'>
label:3
Somewhat positive
